In [1]:
using DynamicPolynomials

println("***Problem setting***")

n=3

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end
function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
v+=star_algebra.(v)
v=v./2

f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{false,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=3
Number of inequality constraints: m=1
Number of equality constraints: l=1


In [2]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [3]:

k=2

println("Relaxation order: k=",k)

Relaxation order: k=2


In [4]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-4,
                                 use_eqcons_to_get_constant_trace=false,
                                 check_tol_each_iter=true)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1


  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=2
  Size of the largest block: s^max=13
  Number of equality trace constraints: zeta=39
Modeling time:
 23.106484 seconds (47.69 M allocations: 2.341 GiB, 6.51% gc time)
iter=1   val=-4.024598317790835   gap=0.018556817341320038   feas=3.043018619114468
iter=2   val=-1.8459532139664194   gap=0.2163849639637823   feas=0.5468203066793152
iter=4   val=-2.123833121213326   gap=0.005868917558758952   feas=0.5391394775506972
iter=8   val=-2.0891843391715352   gap=0.1062669641283489   feas=0.42162502302698185
iter=16   val=-2.038682145901037   gap=0.1627478725161107   feas=0.3986952412566465
iter=32   val=-2.036719250027245   gap=0.3117566478999945   feas=0.39662358066506165
iter=64   val=-1.4523685578094125   gap=0.2664092093237933   feas=0.23140046431279077
iter=128   val=-0.9038855018942465   gap=0.07624105599853252   feas=0.07198062416256897
iter=256   val=-0.6693843829996262   gap=0.0279629645

-0.7039534186382683

In [5]:
using NCTSSOS

@time ctpNCPOP.nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
    n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS=false,QUIET=false);

------------------------------------------------------
The sizes of blocks:
[13]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 73              
  Cones                  : 0               
  Scalar variables       : 11              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Probl

In [6]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)